In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
import pandas as pd

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from kaggle.competitions import twosigmanews

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

# Get 2Sigma environment
env = twosigmanews.make_env()
# Any results you write to the current directory are saved as output.

In [ ]:
from datetime import datetime

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain

%matplotlib inline

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()

In [ ]:
market_train_df.dropna(inplace=True)

In [ ]:
market_train_df = market_train_df[market_train_df['universe'] == 1]

In [ ]:
news_train_df['day'] = news_train_df['time'].apply(lambda x: x.day)
news_train_df['month'] = news_train_df['time'].apply(lambda x: x.month)
news_train_df['year'] = news_train_df['time'].apply(lambda x: x.year)
market_train_df['day'] = market_train_df['time'].apply(lambda x: x.day)
market_train_df['month'] = market_train_df['time'].apply(lambda x: x.month)
market_train_df['year'] = market_train_df['time'].apply(lambda x: x.year)



In [ ]:
new_df = pd.merge(news_train_df, market_train_df,  how='left', left_on=['day','month','year','assetName'], right_on = ['day','month','year','assetName'])

In [ ]:
new_df = new_df.groupby(['day','month','year','assetName']).mean()

In [ ]:
new_df.dropna(inplace=True)

In [ ]:
new_df.head()

In [ ]:
X = new_df.drop(['universe','returnsOpenNextMktres10'],axis=1)
y = new_df['returnsOpenNextMktres10']

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=5)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense


In [ ]:
model = Sequential()
model.add(Dense(units=20,input_shape=(35,),activation='relu'))
model.add(Dense(units=20,activation='relu'))
model.add(Dense(units=20,activation='relu'))
model.add(Dense(1, activation='linear'))

In [ ]:
from keras import optimizers
optimizer = optimizers.adam(0.001)

In [ ]:
model.compile(optimizer=optimizer,loss='mean_squared_error',metrics=['mae'])

In [ ]:
model.fit(x=X_train,y=y_train,epochs=100,batch_size=1000)

In [ ]:
def make_predictions(predictions_template_df, market_obs_df,news_obs_df):
    news_obs_df['day'] = news_obs_df['time'].apply(lambda x: x.day)
    news_obs_df['month'] = news_obs_df['time'].apply(lambda x: x.month)
    news_obs_df['year'] = news_obs_df['time'].apply(lambda x: x.year)
    market_obs_df['day'] = market_obs_df['time'].apply(lambda x: x.day)
    market_obs_df['month'] = market_obs_df['time'].apply(lambda x: x.month)
    market_obs_df['year'] = market_obs_df['time'].apply(lambda x: x.year)
    
    news_obs_df = news_obs_df.groupby(['day','month','year','assetName']).mean()
    # new = pd.merge(market_obs_df, news_obs_df,  how='left', left_on=['day','month','year','assetName'], right_on = ['day','month','year','assetName'])
    new = pd.merge(market_obs_df, news_obs_df, on=['day','month','year','assetName'], how='left')
    
    X = new.drop(['time','assetCode','assetName','day','month','year'],axis=1)

    X = X.fillna(0)
    X = sc.transform(X)
    predictions_template_df.confidenceValue = np.clip((model.predict(X)), -1, 1)

In [ ]:
# news_obs_df['day'] = news_obs_df['time'].apply(lambda x: x.day)
# news_obs_df['month'] = news_obs_df['time'].apply(lambda x: x.month)
# news_obs_df['year'] = news_obs_df['time'].apply(lambda x: x.year)
# market_obs_df['day'] = market_obs_df['time'].apply(lambda x: x.day)
# market_obs_df['month'] = market_obs_df['time'].apply(lambda x: x.month)
# market_obs_df['year'] = market_obs_df['time'].apply(lambda x: x.year)

# news_obs_df = news_obs_df.groupby(['day','month','year','assetName']).mean()
# # new = pd.merge(market_obs_df, news_obs_df,  how='left', left_on=['day','month','year','assetName'], right_on = ['day','month','year','assetName'])
# new = pd.merge(market_obs_df, news_obs_df, on=['day','month','year','assetName'], how='left')

# X = new

In [ ]:
# X = X.drop(['time','assetCode','assetName','day','month','year'],axis=1).head()

In [ ]:
days = env.get_prediction_days()

In [ ]:
count = 1
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    make_predictions(predictions_template_df, market_obs_df,news_obs_df)
    env.predict(predictions_template_df)
    print(count)
    count = count + 1
print('Done!')

In [ ]:
env.predict(predictions_template_df)

In [ ]:
env.write_submission_file()

In [ ]:
days = env.get_prediction_days()
(market_obs_df, news_obs_df, predictions_template_df) = next(days)


In [ ]:
news_obs_df.head()

In [ ]:
news_obs_df['assetCode'] = news_obs_df['assetCodes'] 

In [ ]:
len(df2['assetCode'].unique())

In [ ]:
df2.info()

In [ ]:
 df2 = market_obs_df.merge(news_obs_df,on=['assetName'])


In [ ]:
X2 = df2[['returnsClosePrevRaw1','returnsClosePrevMktres1','returnsOpenPrevMktres1','returnsOpenPrevRaw10','returnsClosePrevRaw10','urgency',
      'sentenceCount','wordCount','firstMentionSentence','relevance','sentimentClass','sentimentNegative','sentimentNeutral','sentimentPositive','sentimentWordCount','noveltyCount12H','noveltyCount24H','noveltyCount3D','noveltyCount5D','noveltyCount7D','volumeCounts12H','volumeCounts24H','volumeCounts3D','volumeCounts5D','volumeCounts7D']]

In [ ]:
X2 = X2.fillna(0)

In [ ]:
X2.head()

In [ ]:
X = mlp.predict(X2)

In [ ]:
df2.shape

In [ ]:
X2 = df2[['returnsClosePrevRaw1','returnsClosePrevMktres1','returnsOpenPrevMktres1','returnsOpenPrevRaw10','returnsClosePrevRaw10','urgency',
      'sentenceCount','wordCount','firstMentionSentence','relevance','sentimentClass','sentimentNegative','sentimentNeutral','sentimentPositive','sentimentWordCount','noveltyCount12H','noveltyCount24H','noveltyCount3D','noveltyCount5D','noveltyCount7D','volumeCounts12H','volumeCounts24H','volumeCounts3D','volumeCounts5D','volumeCounts7D']]

In [ ]:
predictions_template_df['confidenceValue'] = X

In [ ]:
len(predictions_template_df['assetCode'].unique())

In [ ]:
news_obs_df.head()

In [ ]:
env.predict(predictions_template_df)

In [ ]:
def con_value(x):
    if x > 1:
        return 1
    elif x < -1:
        return -1
    else:
        return x

In [ ]:
predictions_template_df['confidenceValue'] = predictions_template_df['confidenceValue'].apply(con_value)

In [ ]:
X.reshape(1832,)

In [ ]:
predictions_template_df.shape

In [ ]:
predictions_template_df['confidenceValue'] = X   

In [ ]:
predictions_template_df['confidenceValue'] = 2.0 * np.random.rand(len(predictions_template_df)) - 1.0

In [ ]:
for (market_obs_df, news_obs_df, predictions_template_df) in days: 
    df2 = market_obs_df.merge(news_obs_df,left_index=True,right_index=True)
    X2 = df2[['returnsClosePrevRaw1','returnsClosePrevMktres1','returnsOpenPrevMktres1','returnsOpenPrevRaw10','returnsClosePrevRaw10','urgency','sentenceCount','wordCount','firstMentionSentence','relevance','sentimentClass','sentimentNegative','sentimentNeutral','sentimentPositive','sentimentWordCount','noveltyCount12H','noveltyCount24H','noveltyCount3D','noveltyCount5D','noveltyCount7D','volumeCounts12H','volumeCounts24H','volumeCounts3D','volumeCounts5D','volumeCounts7D']]
    X2 = X2.fillna(0) 
    X = mlp.predict(X2)
#     predictions_template_df['confidenceValue'] = 2.0 * np.random.rand(len(predictions_template_df)) - 1.0
    predictions_template_df['confidenceValue'] = X    
    predictions_template_df['confidenceValue'] = predictions_template_df['confidenceValue'].apply(con_value)    
    env.predict(predictions_template_df)

In [ ]:
env.write_submission_file()